In [1]:
!pip install jieba

In [2]:
!pip install gensim

In [3]:
!pip install plotly

In [4]:
!pip install requests

In [5]:
!pip install feedparser

In [6]:
!pip install lxml

In [7]:
document = "科技大廠蘋果本週爆出重啟電動車計畫，據稱5年內上市、搭配突破性降低成本、提升續航里程的電池技術，引發市場話題"

In [8]:
text_corpus = [
    "兩項最新研究指出，具有2019冠狀病毒疾病（COVID-19）抗體的民眾，再度染疫的可能性相當低。研究人員還發現，抗體保護力能維持6個月，甚至是更久時間。",
    "由俄國富豪尤里·米爾納注資 1 億美元、已故英國天文物理學家霍金啟動的突破聆聽（Breakthrough Listen）計畫，近日收到了一個有趣的無線電訊號，來自距離太陽最近的另一顆恆星比鄰星。",
    "科技大廠蘋果本週爆出重啟電動車計畫，據稱5年內上市、搭配突破性降低成本、提升續航里程的電池技術，引發市場話題",
    "中國監管單位對阿里巴巴集團展開反壟斷調查，使阿里巴巴美國存託憑證（ADR）在週四（24日）創下有史以來最大跌幅。",
    "日本就有一名高齡95歲的阿嬤相當喜歡用GB遊戲主機玩「俄羅斯方塊」，無奈電子產品終究無法抵抗時間的摧殘而損壞，沒想到阿嬤親筆寫信給任天堂官方後，竟然得到一台全新的遊戲主機，讓網友感動不已。"
]

In [9]:
len(text_corpus)

5

### 準備資料

In [10]:
import feedparser

In [11]:
rss_url = "https://money.udn.com/rssfeed/news/1001/5588/10511?ch=money"

In [12]:
newsFeed = feedparser.parse(rss_url)

In [13]:
i = 1
for e in newsFeed['entries']:
    title = e['title']
    link_url = e['links'][0]['href']
    print("%s, %s, %s"%(i, title, link_url))
    i = i + 1

1, 喀布爾機場爆炸 五角大廈證實數名美軍傷亡, https://money.udn.com/money/story/10511/5702734
2, 喀布爾機場附近爆炸傳13死逾60傷 疑ISIS-K犯案, https://money.udn.com/money/story/10511/5702719
3, 喀布爾機場疑遭自殺攻擊 造成美軍等人員傷亡, https://money.udn.com/money/story/10511/5702566
4, 網安躍升拜登首要議題  攜手科技巨擘研擬新規範, https://money.udn.com/money/story/10511/5701435
5, 貴森森總部逃 胡佛報告：企業速離加州前進德州, https://money.udn.com/money/story/10511/5701200
6, 美眾院查國會之亂 首向政府索川普親信通聯等資料, https://money.udn.com/money/story/10511/5701088
7, 世衛國際專家組：疫情溯源機會之窗正變小, https://money.udn.com/money/story/10511/5701086
8, 阿富汗的眼淚（下）／台商眼中的阿富汗零商機 大陸有機會取得當地富礦？, https://money.udn.com/money/story/10511/5700568
9, 阿富汗的眼淚（上）／美國急撤阿富汗，最想推大陸入坑？, https://money.udn.com/money/story/10511/5700287
10, 彭博：華爾街和北京政府官員將恢復對話 以謀求共識, https://money.udn.com/money/story/10511/5698328
11, 美國准買汽車晶片！華為業務轉向有宏大企圖, https://money.udn.com/money/story/10511/5698185
12, 美疫情調查報告到期 真相恐難水落石出, https://money.udn.com/money/story/10511/5697815
13, 搞砸阿富汗撤軍 拜登最新民調支持度大失血, https://money.udn.com/money/story/10511/5697681
14, 斷芯周年前夕

In [14]:
import requests
from lxml import etree
from io import StringIO
import jieba
import pandas as pd
import time
import random
import pprint

each_article_text_list = []
news_crawl_result_file = 'news_crawl_result.txt'

In [15]:
#設定 request header
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36"
my_headers = {
    'User-Agent': user_agent,
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7",
    "accept-encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "cache-control": "no-cache",
    "Accept-Charset": "UTF8,utf-8;q=0.7,*;q=0.7"
}

In [16]:
#爬文
for e in newsFeed['entries']:
    url = e['links'][0]['href']
    print(url)
    r = requests.get(url, headers = my_headers)
    if r.status_code == 200:
        parse_tree = etree.parse(StringIO(r.text), etree.HTMLParser())
        article_elements = parse_tree.xpath('//*[@id="article_body"]//p')
        post = []
        for a_part in article_elements:
            if type(a_part.text) is str:
                #each_article_text_list.append(a_part.text.strip())
                post.append(a_part.text.strip())
        each_article_text_list.append(post)
        #print(each_article_text_list)
        sleep_time = random.randint(3,10)
        print("sleep time: %s sec"%(sleep_time))
        time.sleep(sleep_time)
    #all_article_text = ''.join(each_article_text_list)
#all_article_text

https://money.udn.com/money/story/10511/5702734
sleep time: 10 sec
https://money.udn.com/money/story/10511/5702719
sleep time: 8 sec
https://money.udn.com/money/story/10511/5702566
sleep time: 10 sec
https://money.udn.com/money/story/10511/5701435
sleep time: 8 sec
https://money.udn.com/money/story/10511/5701200
sleep time: 5 sec
https://money.udn.com/money/story/10511/5701088
sleep time: 10 sec
https://money.udn.com/money/story/10511/5701086
sleep time: 9 sec
https://money.udn.com/money/story/10511/5700568
sleep time: 8 sec
https://money.udn.com/money/story/10511/5700287
sleep time: 7 sec
https://money.udn.com/money/story/10511/5698328
sleep time: 3 sec
https://money.udn.com/money/story/10511/5698185
sleep time: 5 sec
https://money.udn.com/money/story/10511/5697815
sleep time: 10 sec
https://money.udn.com/money/story/10511/5697681
sleep time: 9 sec
https://money.udn.com/money/story/10511/5697630
sleep time: 8 sec
https://money.udn.com/money/story/10511/5697063
sleep time: 6 sec
https:

In [17]:
# 存檔
with open(news_crawl_result_file, 'wb') as saveFile:
    for post in each_article_text_list:
        if len(post) > 0:
            for line in post:
                line = line.encode('utf-8')
                saveFile.write(line)
                saveFile.write('\n'.encode())

In [18]:
import jieba
import time
import multiprocessing
cores = multiprocessing.cpu_count() # Count the number of cores in a computer

In [28]:
!wget https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big -O /home/jovyan/work/jieba_data/dict.txt.big

--2021-08-27 02:50:18--  https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big
Resolving github.com (github.com)... 52.69.186.44
Connecting to github.com (github.com)|52.69.186.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/fxsjy/jieba/master/extra_dict/dict.txt.big [following]
--2021-08-27 02:50:18--  https://raw.githubusercontent.com/fxsjy/jieba/master/extra_dict/dict.txt.big
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8583143 (8.2M) [text/plain]
Saving to: ‘/home/jovyan/work/jieba_data/dict.txt.big’

/home/jovyan/work/j 100%[===================>]   8.18M  --.-KB/s    in 0.06s   

2021-08-27 02:50:18 (135 MB/s) - ‘/home/jovyan/work/jieba_data/dict.txt.big’ saved [8583143/8

In [29]:
!wget https://raw.githubusercontent.com/fxsjy/jieba/master/extra_dict/stop_words.txt -O /home/jovyan/work/jieba_data/stop_words.txt

--2021-08-27 02:50:21--  https://raw.githubusercontent.com/fxsjy/jieba/master/extra_dict/stop_words.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 222 [text/plain]
Saving to: ‘/home/jovyan/work/jieba_data/stop_words.txt’

/home/jovyan/work/j 100%[===================>]     222  --.-KB/s    in 0s      

2021-08-27 02:50:21 (13.9 MB/s) - ‘/home/jovyan/work/jieba_data/stop_words.txt’ saved [222/222]



In [30]:
jieba.set_dictionary('/home/jovyan/work/jieba_data/dict.txt.big')

Exception: jieba: file does not exist: /home/jieba_data/dict.txt.big

In [ ]:
jieba.load_userdict('/home/jovyan/work/jieba_data/mydict.txt')

Building prefix dict from /home/jovyan/work/jieba_data/dict.txt.big ...
Loading model from cache /tmp/jieba.ud3c40470d3518910bfd0dc86d865fad4.cache
Loading model cost 1.370 seconds.
Prefix dict has been built successfully.


In [ ]:
# 開檔後一次讀一行
stop_words_list = []
with open(file='/home/jovyan/work/jieba_data/stop_words.txt',mode='r', encoding="UTF-8") as file:
    for line in file:
        line = line.strip()
        stop_words_list.append(line)
stop_words_list

['the',
 'of',
 'is',
 'and',
 'to',
 'in',
 'that',
 'we',
 'for',
 'an',
 'are',
 'by',
 'be',
 'as',
 'on',
 'with',
 'can',
 'if',
 'from',
 'which',
 'you',
 'it',
 'this',
 'then',
 'at',
 'have',
 'all',
 'not',
 'one',
 'has',
 'or',
 'that',
 '的',
 '了',
 '和',
 '是',
 '就',
 '都',
 '而',
 '及',
 '與',
 '著',
 '或',
 '一個',
 '沒有',
 '我們',
 '你們',
 '妳們',
 '他們',
 '她們',
 '是否']

### 分詞

In [ ]:
seg_result = []
for line in text_corpus:
    seg_result.append(' '.join(jieba.lcut(line)))
#seg_result

In [ ]:
seg_result = []
for post in each_article_text_list:
    for line in post:
        seg_result.append(' '.join(jieba.lcut(line)))
#seg_result

In [ ]:
seg_stop_result = []
for line in seg_result:
    #print("[%s]"%(line))
    new_line = []
    for word in line.split():
        if word not in stop_words_list:
            #print(word)
            new_line.append(word)
    #print(new_line)
    seg_stop_result.append(' '.join(new_line))

In [ ]:
'（' not in stop_words_list

True

In [ ]:
'妳' not in stop_words_list

True

In [ ]:
len(seg_stop_result)

653

In [ ]:
seg_stop_result

['經濟日報 全新 podcast 上線 囉 ！ 想 知道 更 多 內幕 ？',
 '【 經濟 尬 你講 】 精彩 新聞 白話 聊 ， 財經新聞 讓 記者 尬 你講 。',
 '',
 '拜登 政府 上台 後 ， 持續 推展 川普 遺留 「 印太 戰略 」 ， 從 國際 戰場 一路 延伸 到 科技 場域 ， 想 建構 綿密 「 抗中 包圍網 」 ； 中國 大陸 方面 呢 ， 則是 高舉 「 科技 自主 」 大旗 ， 近期 更 揮刀 在 美 上市 滴滴 打車 ， 試圖 脫鉤 華爾街 。',
 '',
 '🎧 立即 收聽 經濟 尬 你講 ／ 拜登 築起 小院 高牆 大陸 加速 高科技 自主 4QA 解析 美 中 科技 戰 按 右下角 播放 鍵 ↓',
 '',
 '',
 '',
 '【 本集 重點 】',
 '( 00 : 58 ) ： 科技 戰場 也 要 圍堵 中國 大陸 ， 雙方 如何 應對 ？',
 '( 14 : 28 ) ： 美國 怕 ？ 用 創新 競爭法 抗 中能 成 ？',
 '( 17 : 55 ) ： 抗中 法案 還沒過 ， 中國 大陸 超前 部署 抗美 法案',
 '( 21 : 59 ) ： 台商 好 尷尬 ， 到底 該選 哪邊 站 ？',
 '( 26 : 34 ) ： 大家 來 搶 台積 電是 好 壞 ？ 台灣 該 怎麼辦 ？',
 '',
 '【 更 多 收聽 平台 】',
 'Apple Podcast ：',
 'Spotify ：',
 'KKBOX ：',
 'Google Podcasts ：',
 '',
 '',
 '從 印太 戰略 到 科技 戰 ， 現在 又 來到 疫苗 戰 ， 美國 總統 拜登 6 月 10 日 在 英國 參加 G7 峰會時 表示 ， 今 、 明年 將 無條件 捐贈 5 億支 輝瑞 BNT 新冠 疫苗 ， 現在 美國 ， 就是 疫苗 軍火庫 ， 中國 大陸 方面 ， 在 北京 大興 ， 也 悄悄 誕生 一座 新冠 疫苗 生產 工廠 ， 年 產能 設定 為 10 億支 。 大陸 國藥 集團 董事長 劉敬楨 稱 它 「 全球 最大 新冠 疫苗 『 兵廠 』 」 ， 兩邊 又 開啟 疫苗 霸權 之戰 。',
 '',
 '🎧 立即 收聽 經濟 尬 你講 ／ 短兵相接 軍火庫 對 上 兵工廠 4QA 解析 美 中 

In [ ]:
#製作 sentence 的時候的樣子
for line in seg_stop_result:
    print(line.split())

['經濟日報', '全新', 'podcast', '上線', '囉', '！', '想', '知道', '更', '多', '內幕', '？']
['【', '經濟', '尬', '你講', '】', '精彩', '新聞', '白話', '聊', '，', '財經新聞', '讓', '記者', '尬', '你講', '。']
[]
['拜登', '政府', '上台', '後', '，', '持續', '推展', '川普', '遺留', '「', '印太', '戰略', '」', '，', '從', '國際', '戰場', '一路', '延伸', '到', '科技', '場域', '，', '想', '建構', '綿密', '「', '抗中', '包圍網', '」', '；', '中國', '大陸', '方面', '呢', '，', '則是', '高舉', '「', '科技', '自主', '」', '大旗', '，', '近期', '更', '揮刀', '在', '美', '上市', '滴滴', '打車', '，', '試圖', '脫鉤', '華爾街', '。']
[]
['🎧', '立即', '收聽', '經濟', '尬', '你講', '／', '拜登', '築起', '小院', '高牆', '大陸', '加速', '高科技', '自主', '4QA', '解析', '美', '中', '科技', '戰', '按', '右下角', '播放', '鍵', '↓']
[]
[]
[]
['【', '本集', '重點', '】']
['(', '00', ':', '58', ')', '：', '科技', '戰場', '也', '要', '圍堵', '中國', '大陸', '，', '雙方', '如何', '應對', '？']
['(', '14', ':', '28', ')', '：', '美國', '怕', '？', '用', '創新', '競爭法', '抗', '中能', '成', '？']
['(', '17', ':', '55', ')', '：', '抗中', '法案', '還沒過', '，', '中國', '大陸', '超前', '部署', '抗美', '法案']
['(', '21', ':', '59', ')', '：', '台商', '好

### word2vec 建模

In [ ]:
from gensim.models import word2vec

/opt/conda/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [ ]:
class MySentence:
    def __init__(self, seg_result_list):
        self.seg_result_list = seg_result_list
        
    def __iter__(self):
        for line in self.seg_result_list:
            yield line.split()

In [ ]:
sentence = MySentence(seg_stop_result)

In [ ]:
start_time = time.time()
model = word2vec.Word2Vec( sentence , vector_size=300, sg=1, window=5, min_count=1, workers=cores-1)
print("--- spend %s seconds ---" % (time.time() - start_time))

--- spend 0.5286285877227783 seconds ---


### 視覺化

In [ ]:
from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction
import numpy as np 

In [ ]:
len(model.wv.vectors)

3301

In [ ]:
def reduce_dimensions(model):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    # extract the words & their vectors, as numpy arrays
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  # fixed-width numpy strings

    # reduce using t-SNE
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels


x_vals, y_vals, labels = reduce_dimensions(model)

In [ ]:
def plot_with_plotly(x_vals, y_vals, labels, plot_in_notebook=True):
    from plotly.offline import init_notebook_mode, iplot, plot
    import plotly.graph_objs as go

    trace = go.Scatter(x=x_vals, y=y_vals, mode='text', text=labels)
    data = [trace]

    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')


def plot_with_matplotlib(x_vals, y_vals, labels):
    import matplotlib.pyplot as plt
    import random

    random.seed(0)

    plt.figure(figsize=(12, 12))
    plt.scatter(x_vals, y_vals)

    #
    # Label randomly subsampled 25 data points
    #
    indices = list(range(len(labels)))
    selected_indices = random.sample(indices, 25)
    for i in selected_indices:
        plt.annotate(labels[i], (x_vals[i], y_vals[i]))

try:
    get_ipython()
except Exception:
    plot_function = plot_with_matplotlib
else:
    plot_function = plot_with_plotly

plot_function(x_vals, y_vals, labels)

In [ ]:
model.wv.similar_by_word('拜登')

[('在', 0.9995190501213074),
 ('美國', 0.9995179176330566),
 ('年', 0.9995173215866089),
 ('阿富汗', 0.999514102935791),
 ('大陸', 0.9995139837265015),
 ('，', 0.9994901418685913),
 ('但', 0.9994772672653198),
 ('。', 0.9994750618934631),
 ('不', 0.9994539618492126),
 ('為', 0.9994365572929382)]

In [ ]:
model.wv.similar_by_word('阿富汗')

[('在', 0.999652624130249),
 ('。', 0.9996415972709656),
 ('，', 0.9996364116668701),
 ('美國', 0.9996116161346436),
 ('年', 0.9995957612991333),
 ('美元', 0.9995949268341064),
 ('經濟', 0.9995834827423096),
 ('到', 0.9995802044868469),
 ('中', 0.9995796084403992),
 ('但', 0.9995783567428589)]

## 潛在語意分析 

In [ ]:
len(seg_stop_result)

653

In [ ]:
processed_corpus = []
for post in seg_stop_result:
    post_corpus = []
    for text in post.split():
        post_corpus.append(text)
    processed_corpus.append(post_corpus)

In [ ]:
len(processed_corpus)

653

In [ ]:
#一個句子一個 list, 全部的句子再用一個 list 包起來 
processed_corpus

[['經濟日報', '全新', 'podcast', '上線', '囉', '！', '想', '知道', '更', '多', '內幕', '？'],
 ['【',
  '經濟',
  '尬',
  '你講',
  '】',
  '精彩',
  '新聞',
  '白話',
  '聊',
  '，',
  '財經新聞',
  '讓',
  '記者',
  '尬',
  '你講',
  '。'],
 [],
 ['拜登',
  '政府',
  '上台',
  '後',
  '，',
  '持續',
  '推展',
  '川普',
  '遺留',
  '「',
  '印太',
  '戰略',
  '」',
  '，',
  '從',
  '國際',
  '戰場',
  '一路',
  '延伸',
  '到',
  '科技',
  '場域',
  '，',
  '想',
  '建構',
  '綿密',
  '「',
  '抗中',
  '包圍網',
  '」',
  '；',
  '中國',
  '大陸',
  '方面',
  '呢',
  '，',
  '則是',
  '高舉',
  '「',
  '科技',
  '自主',
  '」',
  '大旗',
  '，',
  '近期',
  '更',
  '揮刀',
  '在',
  '美',
  '上市',
  '滴滴',
  '打車',
  '，',
  '試圖',
  '脫鉤',
  '華爾街',
  '。'],
 [],
 ['🎧',
  '立即',
  '收聽',
  '經濟',
  '尬',
  '你講',
  '／',
  '拜登',
  '築起',
  '小院',
  '高牆',
  '大陸',
  '加速',
  '高科技',
  '自主',
  '4QA',
  '解析',
  '美',
  '中',
  '科技',
  '戰',
  '按',
  '右下角',
  '播放',
  '鍵',
  '↓'],
 [],
 [],
 [],
 ['【', '本集', '重點', '】'],
 ['(',
  '00',
  ':',
  '58',
  ')',
  '：',
  '科技',
  '戰場',
  '也',
  '要',
  '圍堵',
  '中國',
  '大陸',
  '，',
  '雙方'

In [ ]:
from gensim import corpora
#建立辭典
dictionary = corpora.Dictionary(processed_corpus, prune_at=100)
print(dictionary)

Dictionary(3350 unique tokens: ['podcast', '上線', '內幕', '全新', '囉']...)


In [ ]:
#pprint.pprint(dictionary.token2id)

### 每個()中的第一個數字對應於字典中token 的ID，第二個數字對應於此token的計數。

In [ ]:
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus ]
#pprint.pprint(bow_corpus)

In [ ]:
new_text = "拜登 阿富汗"
#new_text = "台灣"

In [ ]:
new_vec = dictionary.doc2bow(new_text.split())
print(new_vec)

[(51, 1), (370, 1)]


## 建立 tf-idf 模型

In [ ]:
from gensim import models

In [ ]:
tfidf = models.TfidfModel(bow_corpus)

In [ ]:
corpus_tfidf = tfidf[bow_corpus]

In [ ]:
for i in corpus_tfidf:
    pprint.pprint(i)

[(0, 0.3508500048475304),
 (1, 0.3508500048475304),
 (2, 0.3508500048475304),
 (3, 0.3508500048475304),
 (4, 0.3508500048475304),
 (5, 0.1914665076663238),
 (6, 0.2382891172075229),
 (7, 0.18353081105962057),
 (8, 0.25386144698176144),
 (9, 0.2758094130875254),
 (10, 0.29138174286176394),
 (11, 0.17882085522175648)]
[(12, 0.04153351435038293),
 (13, 0.21767891336617443),
 (14, 0.21767891336617443),
 (15, 0.45993833532260686),
 (16, 0.45993833532260686),
 (17, 0.22996916766130343),
 (18, 0.2769037030128168),
 (19, 0.2769037030128168),
 (20, 0.14683666530013958),
 (21, 0.2769037030128168),
 (22, 0.2769037030128168),
 (23, 0.1448492532341269),
 (24, 0.2769037030128168),
 (25, 0.040171752683377596)]
[]
[(6, 0.12234808991047605),
 (7, 0.09423277250764917),
 (12, 0.027019940537054213),
 (25, 0.15680421737739655),
 (26, 0.19497624891297374),
 (27, 0.19766611713219825),
 (28, 0.126059313816602),
 (29, 0.1416126563648323),
 (30, 0.1801417892735448),
 (31, 0.05416772980136496),
 (32, 0.094232772

In [ ]:
tfidf[new_vec]

[(51, 0.5980971129334391), (370, 0.8014236354766965)]

## 建立LSI 模型

In [ ]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary)

In [ ]:
#LSI(潛在語意索引)
corpus_lsi = lsi[corpus_tfidf]

In [ ]:
#目前的主題
lsi.print_topics()

[(0,
  '-0.535*"，" + -0.197*"閱讀" + -0.194*"延伸" + -0.193*"》" + -0.188*"。" + -0.184*"美國" + -0.160*"中國" + -0.142*"在" + -0.142*"、" + -0.134*"人民幣"'),
 (1,
  '-0.557*"閱讀" + -0.536*"》" + -0.536*"延伸" + 0.192*"，" + 0.068*"。" + 0.066*"美國" + 0.058*"中國" + 0.052*"、" + 0.050*"在" + 0.050*"人民幣"'),
 (2,
  '-0.327*"人民幣" + 0.245*"大陸" + -0.198*"美元" + 0.151*"政府" + 0.144*"（" + 0.140*"關稅" + 0.136*"）" + 0.126*"拜登" + 0.118*"官員" + -0.116*"有"'),
 (3,
  '0.572*"：" + 0.251*"．" + 0.245*"億美元" + 0.203*"KKBOX" + 0.203*"Spotify" + 0.144*"摘要" + 0.139*"以下" + 0.132*"答" + 0.128*"？" + 0.127*":"'),
 (4,
  '-0.217*"）" + -0.211*"（" + -0.200*"、" + 0.172*")" + 0.171*":" + 0.170*"中國" + -0.165*"」" + 0.165*"(" + 0.164*"？" + -0.164*"「"'),
 (5,
  '-0.392*"人民幣" + 0.211*"「" + 0.204*"」" + -0.199*"大陸" + -0.167*"美元" + 0.127*"金融" + -0.124*"關稅" + 0.120*"打" + -0.109*"國際化" + 0.101*"阿富汗"'),
 (6,
  '0.285*"？" + 0.251*":" + 0.240*"(" + 0.240*")" + 0.237*"人民幣" + 0.198*"疫苗" + 0.193*"」" + 0.190*"「" + -0.149*"億美元" + -0.149*"．"'),
 (7,
  '0.352*"【" +

In [ ]:
#新文章
new_doc = '拜登近期因對撤離過程的處理而受到猛烈批評。最近幾天，撤離過程凸顯出喀布爾機場的混亂場面，人群拼命想離開這個國家。但拜登辯護，稱結束美國在阿富汗20年的參與不可避免的會出現問題。'
#new_vec_bow = dictionary.doc2bow()

In [ ]:
#分詞
new_seg_result = jieba.lcut(new_doc)
len(new_seg_result)

50

In [ ]:
new_doc_bow = dictionary.doc2bow(new_seg_result)

In [ ]:
new_doc_lsi = lsi[new_doc_bow]

In [ ]:
pprint.pprint(new_doc_lsi)

[(0, -3.2666650702204376),
 (1, 1.1665182770118234),
 (2, 0.3347842584658224),
 (3, -0.4097299415714869),
 (4, -0.37528041088989406),
 (5, 0.534030025087737),
 (6, -0.6350032439186278),
 (7, -0.49479378117449624),
 (8, -0.3615027871917749),
 (9, 0.4612790969907953),
 (10, 0.6854763216655945),
 (11, 0.21917369800110154),
 (12, -0.17942954296652452),
 (13, 0.132587311941002),
 (14, -0.14276793913707386),
 (15, 0.11348283746123324),
 (16, -0.29201824302565804),
 (17, 0.5068202266063987),
 (18, -0.33156373232999503),
 (19, 0.029644964389186707),
 (21, 0.0703571889551207),
 (22, 0.1560690608191833),
 (23, -0.10460658704140001),
 (24, -0.06715583551728678),
 (25, -0.0033178630828204214),
 (27, 0.016124318532393968),
 (28, 0.416879264390548),
 (29, 0.0259579011384968),
 (30, 0.40915476445138194),
 (31, 0.2153972264679058),
 (32, 0.09858410853194173),
 (33, 0.004803479220109191),
 (34, 0.025058762837632056),
 (35, -0.3212113249044948),
 (36, -0.17866067481539732),
 (37, -0.15730727840784406),


### 建立索引

In [ ]:
from gensim import similarities

In [ ]:
index = similarities.MatrixSimilarity(corpus_lsi)

In [ ]:
index

In [ ]:
#新文章相似度
sims = index[new_doc_lsi]

In [ ]:
sims

array([ 5.88316880e-02,  6.03402890e-02,  0.00000000e+00,  2.48665899e-01,
        0.00000000e+00,  2.77439822e-02,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -2.82117818e-03,  4.07916978e-02,  1.57680959e-02,
        2.77444813e-02,  1.33228013e-02, -5.04836440e-03,  0.00000000e+00,
       -8.64064088e-04, -2.29117926e-04, -3.38344777e-04, -3.38344777e-04,
       -2.29117926e-04,  0.00000000e+00,  0.00000000e+00,  2.48157427e-01,
        0.00000000e+00,  1.87674705e-02,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  6.42978176e-02,  2.68176347e-02,  3.27843614e-02,
        5.48992679e-02,  8.10812116e-02,  0.00000000e+00, -8.64064088e-04,
       -2.29117926e-04, -3.38344777e-04, -3.38344777e-04, -2.29117926e-04,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  1.52754351e-01,  0.00000000e+00,  1.45041481e-01,
        0.00000000e+00,  1.23670034e-01,  0.00000000e+00,  1.65871680e-01,
        3.36099297e-01,  

In [ ]:
print(list(enumerate(sims))) # print (document_number, document_similarity) 2-tuples

[(0, 0.058831688), (1, 0.06034029), (2, 0.0), (3, 0.2486659), (4, 0.0), (5, 0.027743982), (6, 0.0), (7, 0.0), (8, 0.0), (9, -0.0028211782), (10, 0.040791698), (11, 0.015768096), (12, 0.027744481), (13, 0.013322801), (14, -0.0050483644), (15, 0.0), (16, -0.0008640641), (17, -0.00022911793), (18, -0.00033834478), (19, -0.00033834478), (20, -0.00022911793), (21, 0.0), (22, 0.0), (23, 0.24815743), (24, 0.0), (25, 0.01876747), (26, 0.0), (27, 0.0), (28, 0.0), (29, 0.06429782), (30, 0.026817635), (31, 0.03278436), (32, 0.054899268), (33, 0.08108121), (34, 0.0), (35, -0.0008640641), (36, -0.00022911793), (37, -0.00033834478), (38, -0.00033834478), (39, -0.00022911793), (40, 0.0), (41, 0.0), (42, 0.0), (43, 0.0), (44, 0.0), (45, 0.15275435), (46, 0.0), (47, 0.14504148), (48, 0.0), (49, 0.123670034), (50, 0.0), (51, 0.16587168), (52, 0.3360993), (53, 0.0), (54, 0.24645673), (55, 0.0), (56, 0.29113927), (57, 0.18631826), (58, 0.0), (59, 0.2727686), (60, 0.0), (61, 0.2043788), (62, 0.0), (63, 0.2

In [ ]:
sims = sorted(enumerate(sims), key=lambda i: -i[1])

In [ ]:
for doc_pos, doc_score in sims[:5]:
    print(doc_score, seg_stop_result[doc_pos])

0.4281414 法新社 報導 ， 一些 分析家 認為 ， 美軍 撤離 後 阿富汗 政府 潰敗 ， 將 不可避免 地 削弱 美國 在 全球 舞台 聲量 ， 尤其 在 美國 總統 拜登 面對 崛起 中國 談到 民主 國家 團結 之際 。
0.4053515 答 ： 我 覺得 騎虎難下 ， 兩邊 有 這個 問題 ， 騎虎難下 。 怎麼 擺脫 這樣 僵局 ， 很難說 。 對 中國 來說 ， 中國 考慮 得 更 多 ， 比如說 ， 戰略 競爭 ， 美國 覺得 中國 挑戰 美國 ， 打擊 這個 、 打擊 那個 ， 最終 兩個 綁 在 一起 ， 爆 的話 ， 很慘 。
0.36261365 答 ： 香港 會 做點事 情 ， 意思 意思 ， 不是 真的 ， 拜登 現在 在乎 ， 他 做 總統 ， 要 拍拍 支持 他 幫派 馬屁 ， 國際 上 問題 就是 吹 吹牛 ， 美國 又 回來 ， 沒 別的 。 千萬別 把 拜登 變成 偉大 領導 ， 根本 不是 ！
0.36253896 答 ： 我 覺得 他 在 這 上面 的話 什麼 看法 ， 拜登 這個 人 從 歷史 上 來說 對 經濟 問題 、 金融 問題 關心 得 不是 很多 ， 歷史 上 主要 還是 外交 ， 參與 外交 比較 多 。 現在 又 對 種族歧視 這件 事情 比較 重視 ， 我 覺得 在 貨幣 問題 上 ， 他 看法 不 明確 。
0.35018992 以 貿易戰 為例 ， 中國 受到 些 影響 ， 但是 後來 疫情 爆發 之後 ， 反而 對 中國 幾乎 沒什麼 影響 ， 現在 對 美國 出口 增速 非常 之快 ， 貿易順差 非常 大 。 銷往 美國 很多 產品價格 出現 明顯 上漲 ， 受害 美國 老百姓 ， 但是 這個 傷害 也 不過 就是 增加 一些 開支 。 中 美 金融 戰 一旦 打 起來 可能 就是 全球性 金融危機 ， 你 說 美國 想 不想 打 呢 ？


In [ ]:
print(sims[:5])

[(255, 0.4281414), (432, 0.4053515), (362, 0.36261365), (392, 0.36253896), (524, 0.35018992)]
